In [1]:
 #load Raw data from TDT ECoG maschine, plz run ""
import numpy as np
import scipy.io as scio
from scipy import signal
import tdt
import os
import  wave
import matplotlib.pyplot as plt
import json
from scipy.fftpack import fft
from random import shuffle
import h5py
import scipy.io as scio
import scipy.io.wavfile
import math
import mne
import os
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import torch
from torch.utils import data as Data
import torch.nn as nn
import seaborn as sns
# import grad_cam

READ IN DATA

In [11]:
a=np.load('/public/DATA/overt_reading/dataset/HS83/400/0/0_data_block_cue.npy')

In [12]:
a.shape

(60, 401, 200)

In [2]:
PATH='/public/DATA/overt_reading/dataset'
train_num=[0,1,2,3,4]#写成更好看的形式
val_num=[5]
# test_num=[5]

In [33]:
#全频段
class CustomDataset(Data.Dataset):
    def __init__(self, HS, path, freq, elec, num_samples):
        super().__init__()
        self.data = [] # 存Ecog数据，0-59为cue，60-120为read，每一个item的shape为（1, n_freq, n_timePoint)
        self.labels = [] # 存label，cue标记为0，read标记为1

        path_elec = os.path.join(path, f'HS{HS}', str(freq), str(elec))
     
        for num in num_samples: # num为块的个数，tarin，test，valid取得块数4，1，1
            # print(num)
            cue_path = os.path.join(path_elec, f'elec{elec}_{num}_data_block_cue.npy')
            read_path = os.path.join(path_elec, f'elec{elec}_{num}_data_block_read.npy')

            if os.path.exists(cue_path) and os.path.exists(read_path):
                elec_cue = np.load(cue_path) # (n_task, n_freq, n_timePoint) (60, 501, 375)
                elec_read = np.load(read_path)[:,:,:elec_cue.shape[2]]  # Slice first 200 time points
                len_cue=elec_cue.shape[0]
                len_read=elec_read.shape[0]

                for i in range(4): # 绘制前4个word的看和读的数据热图，取150~401的freq
                    ax1=plt.subplot(2,4,i+1)
                    # elec_cue[i]=(elec_cue[i]-np.mean(elec_cue[i],axis=1,keepdims=True))/np.std(elec_cue[i],axis=1,keepdims=True)
                    sns.heatmap(np.log10(abs(elec_cue[i][150:,:])+1e-10),cmap='coolwarm',vmin=-1,vmax=1,cbar=False)
                    ax1.set_xticks([])  # 清除x轴刻度
                    ax1.set_yticks([])
                    ax2=plt.subplot(2,4,i+5)
                    # elec_read[i]=(elec_read[i]-np.mean(elec_read[i],axis=1,keepdims=True))/np.std(elec_read[i],axis=1,keepdims=True)
                    sns.heatmap(np.log10(abs(elec_read[i][150:,:])+1e-10),cmap='coolwarm',vmin=-1,vmax=1,cbar=False)
                    ax2.set_xticks([])  # 清除x轴刻度
                    ax2.set_yticks([])
                plt.show()

                for i in range(len_cue): # 将数据添加到data数组中，并在labels数组中添加看的label=0
                    # elec_cue[i]=(elec_cue[i]-np.mean(elec_cue[i],axis=1,keepdims=True))/np.std(elec_cue[i],axis=1,keepdims=True)
                    # elec_cue_concat=self.extracting(elec_cue[i],freq)
                    # elec_cue_hg=elec_cue[i][70:151,:]
                    elec_cue_hg=elec_cue[i][150:,:]
                    elec_cue_hg=elec_cue_hg[np.newaxis,:,:]
                    self.data.append(elec_cue_hg)  # Append data
                    # print(np.sum(elec_cue_concat,axis=0),1)
                    # assert np.array_equal(np.sum(elec_cue_concat,axis=0),elec_cue[i])
                    # self.labels.append([1,0])  # Label for cue
                    self.labels.append(0)

                for j in range(len_read): # 将数据添加到data数组中，并在labels数组中添加读的label=1
                    # elec_read[j]=(elec_read[j]-np.mean(elec_read[j],axis=1,keepdims=True))/np.std(elec_read[j],axis=1,keepdims=True)
                    # elec_read_concat=self.extracting(elec_read[j],freq)
                    # elec_read_hg=elec_read[j][70:151,:]
                    elec_read_hg=elec_read[j][150:,:]
                    elec_read_hg=elec_read_hg[np.newaxis,:,:]
                    self.data.append(elec_read_hg)  # Append data
                    # assert np.array_equal(np.sum(elec_read_hg,axis=0),elec_read[j])
                    # self.labels.append([0,1])  # Label for read
                    self.labels.append(1)
            #     print(len(self.data))
            # print(len(self.data))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return a tuple of (data, label) and convert data to a PyTorch tensor
        # print(self.data.shape)
        return torch.tensor(abs(self.data[idx]), dtype=torch.float), torch.tensor(self.labels[idx])
    
    def extracting(self,stft_block,freq):
        f=torch.arange(stft_block.shape[0])

        bands = {
            'else1': (0, 1),
            'delta': (1, 4),
            'theta': (4, 8),
            'alpha': (8, 12),
            'beta': (12, 30),
            'gamma': (30, 150),
            'else2': (150, freq+1)
        }
        band_list=[]
        for _, (low, high) in bands.items():
            band_data=np.zeros_like(stft_block)
            indices = np.where((f >=low) & (f < high))
            band_data[indices, :] = stft_block[indices, :]
            band_list.append(band_data)
        
        return np.stack(band_list)###############2

In [34]:
HS=86
# print(train_num)
train_dataset = CustomDataset(HS, PATH, 500, 7, train_num)
val_dataset = CustomDataset(HS, PATH, 500, 7, val_num)
# test_dataset = CustomDataset(HS, PATH, 500, 7, test_num)
train_loader = Data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = Data.DataLoader(val_dataset, batch_size=1, shuffle=True)
# test_loader = Data.DataLoader(test_dataset, batch_size=1, shuffle=True)

# print(f'{len(train_loader)}/{len(val_loader)}/{len(test_loader)}')
print(f'{len(train_loader)}/{len(val_loader)}') # 300/120

In [29]:
num=0
for batch in train_loader:
    block,gt=batch
    print(block.shape,gt)
    # if torch.equal(gt,torch.tensor([[0, 1]])):
    #     num+=1
    break


torch.Size([2, 1, 351, 375]) tensor([0, 0])


二分类问题
训练256个模型
读取数据集：83号病人的datablock，time，datalabel
datablock和datalabel作为训练集的x和y
6个block按照4：1：1分为train，val，test
抽取不同波段，模型的第一层为一个线性映射
喂给模型
绘制acc曲线

In [209]:
# class LinearMap(nn.Module):


#     def __init__(self):
#         super(LinearMap,self).__init__()

#         self.weights=nn.Parameter(torch.randn(7,1,1))


#     def forward(self, x):
#         # x is expected to be of shape (batch_size, 7, 401, 200)
#         # Apply weights and sum along the channel dimension
#         x_weighted = (x * self.weights).sum(dim=1,keepdim=True)  # Resulting shape: (batch_size, 401, 200)
#         # x_weighted=torch.transpose(x_weighted,1,2)
#   # Resulting shape: (batch_size, 200, out_channels)

#         return x_weighted
# batch_size = 2 # Example batch size
# bands = 7
# freq=401     # Example number of bands
# out_channels = 64  # Desired output channels for LSTM processing
# input_tensor = torch.randn(batch_size, bands, freq, 200)  # Example input tensor

# model = LinearMap()
# output_tensor = model(input_tensor)
# print(output_tensor.shape)  

In [15]:
import torchvision
from torchvision import models
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision.models import ResNet50_Weights
class ECOGRes50(nn.Module):
    def __init__(self):
        super(ECOGRes50, self).__init__()
        # self.conv7to1 = nn.Conv2d(7, 1, kernel_size=1, stride=1,padding=0, bias=False)
        self.res50= models.resnet50()
        self.res50.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_ftrs = self.res50.fc.in_features
        self.res50.fc = nn.Sequential(nn.Linear(num_ftrs,1, bias=False),
                                    )
        
    def forward(self,x):
        # x = self.conv7to1(x)
        x=self.res50(x)
        return x
    
model = ECOGRes50()

# 准备输入数据
# 假设原始数据的形状是 [2, 401, 200]
data = torch.randn(2, 1,401, 200)
outputs = model(data)
print(outputs.shape)



torch.Size([2, 1])


In [16]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch
# import torch.nn as nn

# import torch
# import torch.nn as nn

# class ECOGLSTM(nn.Module):
#     def __init__(self, input_dim, hidden_dim=256, num_layers=4, dropout_rate=0.5):
#         super(ECOGLSTM, self).__init__()
#         # LSTM layer with dropout
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate)
        
#         # Fully connected layer that maps the LSTM output to 2 outputs (for binary classification)
#         self.fc = nn.Linear(hidden_dim, 2)

#     def forward(self, x):
#         # Forward propagate the LSTM
#         out, _ = self.lstm(x)  # Output shape: (batch_size, sequence_length, hidden_dim)
        
#         # Decode the hidden state of the last time step
#         out = self.fc(out[:, -1, :])  # Shape: (batch_size, 2)
        
#         return out


# # Example usage
# input_dim = 200
# hidden_dim = 100  # Example size, adjust based on your needs
# num_layers = 4  # Example, can be adjusted

# model = ECOGLSTM(401, hidden_dim, num_layers)
# # Test input
# test_input =output_tensor # (batch_size, sequence_length, input_dim)
# output = model(test_input)
# # Applying softmax to get probabilities for each class

# print(output)  # Outputs probabilities for each class


In [17]:
class EcogBandRes(nn.Module):
    def __init__(self):
        super(EcogBandRes,self).__init__()
        self.res=ECOGRes50()

    def forward(self,x):
        # x=self.linearmap(x)
        x=self.res(x)

        return x


hidden_dim = 100  # Example size, adjust based on your needs
num_layers = 4 
    
batch_size = 1 # Example batch size
bands = 7
freq=401     # Example number of bands
input_tensor = torch.randn(batch_size,1,401,200)
net=EcogBandRes()
out=net(input_tensor)
print(out)

tensor([[-0.5023]], grad_fn=<MmBackward0>)


In [213]:
# class EcogBandLSTM(nn.Module):
#     def __init__(self,freq,hidden_dim, num_layers):
#         super(EcogBandLSTM,self).__init__()
#         self.linearmap=LinearMap()
#         self.lstm=ECOGLSTM(freq, hidden_dim, num_layers)

#     def forward(self,x):
#         x=self.linearmap(x)
#         x=self.lstm(x)

#         return x

In [18]:
import os
import itertools
import statistics
from typing import Callable

import numpy as np

# from tqdm import tqdm
from matplotlib import pyplot as plt
from tqdm.autonotebook import tqdm  # may raise warning about Jupyter
from tqdm.auto import tqdm  # who needs warnings

import torch, torchvision
from torch import nn
from torch.utils import data as Data
class Solver(object):
    def __init__(self,
                 model: nn.Module,
                 optimizer: torch.optim.Optimizer,
                 criterion: Callable,
                 lr_scheduler = None,
                 recorder: dict = None,
                 device=None,
                 early_stopping_patience=10):
        device = device if device is not None else \
            ('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.device = device
        self.recorder = recorder
        
        self.model = self.to_device(model)
        self.optimizer = optimizer
        self.criterion = criterion
        self.lr_scheduler = lr_scheduler
        #early stop 
        self.best_per = 0 
        self.early_stopping_patience = early_stopping_patience
        self.early_stopping_counter = 0
        self.best_val_loss = float('inf')       

    def _step(self,
              batch: tuple) -> dict:
        raise NotImplementedError()

    def to_device(self, x):
        if isinstance(x, torch.Tensor):
            return x.to(self.device)
        elif isinstance(x, np.ndarray):
            return torch.tensor(x, device=self.device)
        elif isinstance(x, nn.Module):
            return x.to(self.device)
        else:
            raise RuntimeError("Data cannot transfer to correct device.")

    def to_numpy(self, x):
        if isinstance(x, np.ndarray):
            return x
        elif isinstance(x, torch.Tensor):
            return x.detach().cpu().numpy()
        else:
            raise RuntimeError(f"Cannot convert type {type(x)} into numpy array.")

    def train(self,
              epochs: int,
              data_loader,
              *,
              val_loader=None,
              is_plot=True) -> dict:             
        torch.cuda.empty_cache()

        val_loss_epochs = []
        train_loss_epochs = []
        val_acc=[]
        tpr_list_epochs = []  # Collect TPR for each epoch
        fpr_list_epochs = []  # Collect FPR for each epoch

        pbar_train = tqdm(total=len(data_loader)*2-1, unit='block')
        if val_loader is not None:
            pbar_val = tqdm(total=len(val_loader), desc=f'[Validation] waiting', unit='block')

        train_acc=[]
        for epoch in range(epochs):

            pbar_train.reset()
            pbar_train.set_description(desc=f'[Train] Epoch {epoch + 1}/{epochs}')
            epoch_loss_acc = 0
            epoch_size = 0
            epoch_acc=0
    
            
            for batch in data_loader:
                self.model.train()
                # forward
                step_dict = self._step(batch)
                batch_size = step_dict['batch_size']
                loss = step_dict['loss']
                acc=step_dict['acc']
                # backward
                self.optimizer.zero_grad()
                loss.backward()

                # optimize
                self.optimizer.step()

                # update information
                loss_value = loss.item()
                epoch_loss_acc += loss_value
                epoch_size += batch_size
                epoch_acc+=acc

                pbar_train.update(batch_size)
                pbar_train.set_postfix(loss=loss_value / batch_size)

            epoch_avg_loss = epoch_loss_acc / epoch_size
            epoch_avg_acc=epoch_acc/epoch_size
            pbar_train.set_postfix(epoch_avg_loss=epoch_avg_loss)
            train_loss_epochs.append(epoch_avg_loss)
            train_acc.append(epoch_avg_acc)


            if self.lr_scheduler:
                self.lr_scheduler.step()

            # validate if `val_loader` is specified
            if val_loader is not None:
                pbar_val.reset()
                pbar_val.set_description(desc=f'[Validation] Epoch {epoch + 1}/{epochs}')
                val_avg_loss,total_avg_acc,TP,TN,FP,FN, tpr_list, fpr_list= self.validate(val_loader, pbar=pbar_val,is_test=False)
                val_loss_epochs.append(val_avg_loss)
                val_acc.append(total_avg_acc)

                tpr_list_epochs.append(tpr_list)
                fpr_list_epochs.append(fpr_list)
                if total_avg_acc > self.best_per:
                    torch.save(self.model,'sig_else2_model.pth')
                
                if total_avg_acc > self.best_per:
                    torch.save(self.model,'best_model.pth')
                    self.best_per = total_avg_acc

            # Early Stopping

                if val_avg_loss < self.best_val_loss:
                    self.best_val_loss = val_avg_loss
                    self.early_stopping_counter = 0
                else:
                    self.early_stopping_counter += 1

                if self.early_stopping_counter >= self.early_stopping_patience:
                    print(f'Early stopping at epoch {epoch + 1}\n')
                    print('TP:',TP,'TN:',TN,'FP:',FP,'FN:',FN,'\n')
                    break

        pbar_train.close()
        if val_loader is not None:
            pbar_val.close()
        train_loss_epochs = torch.tensor(train_loss_epochs).numpy()
        val_loss_epochs = torch.tensor(val_loss_epochs).numpy()

        plt.figure()
        plt.plot(list(range(1, epochs + 1)), train_loss_epochs, label='train')
        if val_loader is not None:
            plt.plot(list(range(1, epochs + 1)), val_loss_epochs, label='validation')
        plt.legend()
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.show()
        plt.close('all')

        if val_loader is not None:
            train_acc=torch.tensor(train_acc).numpy()
            val_acc=torch.tensor(val_acc).numpy()
            plt.figure()
            plt.plot(list(range(1, epochs + 1)), train_acc, label='train')
            plt.plot(list(range(1, epochs + 1)), val_acc, label='validation')
            plt.legend()
            plt.xlabel('Epochs')
            plt.ylabel('ACC')
            plt.show()
            plt.close('all')

        all_tpr = list(itertools.chain.from_iterable(tpr_list_epochs))
        all_fpr = list(itertools.chain.from_iterable(fpr_list_epochs))

        # Plot ROC curve
        plt.figure()
        plt.plot(all_fpr, all_tpr, marker='.')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.show()

        # Calculate AUC
        auc = np.trapz(all_tpr, all_fpr)
        print(f'AUC: {auc}')
        
    def validate(self, data_loader, *, pbar=None,is_test=False) -> float:
        """
        :param pbar: when pbar is specified, do not print average loss
        :return:
        """       
        torch.cuda.empty_cache()

        metrics_acc = {}
        loss_acc = 0
        size_acc = 0
        total_acc=0
        is_need_log = (pbar is None)  

        # Initialize TP, TN, FP, FN
        TP, TN, FP, FN = 0, 0, 0, 0  

        # Initialize lists to collect TPR and FPR
        tpr_list = []
        fpr_list = []
    
        with torch.no_grad():
            if pbar is None:
                pbar = tqdm(total=len(data_loader), desc=f'[Validation]', unit='block')

            
            for batch in data_loader:
                self.model.eval()

                # forward
                pred = self.model(batch[0].to(self.device))
                pred_mask=torch.sigmoid(pred)

                pred_mask[pred_mask>0.5]=1
                pred_mask[pred_mask<=0.5]=0
                acc=torch.sum(pred_mask==batch[1].to(torch.int).to(self.device))

                TP += ((pred_mask == 1) & (batch[1].to(self.device) == 1)).sum().item()
                TN += ((pred_mask == 0) & (batch[1].to(self.device) == 0)).sum().item()
                FP += ((pred_mask == 1) & (batch[1].to(self.device) == 0)).sum().item()
                FN += ((pred_mask == 0) & (batch[1].to(self.device) == 1)).sum().item()

                # Calculate TPR and FPR for each batch
                tpr = TP / (TP + FN)
                fpr = FP / (FP + TN)
                tpr_list.append(tpr)
                fpr_list.append(fpr)

                step_dict = self._step(batch)
                batch_size = step_dict['batch_size']
                loss = step_dict['loss']
                acc=step_dict['acc']
                # print(acc)
                loss_value = loss.item()

                # aggregate metrics
                metrics_acc = self._aggregate_metrics(metrics_acc, step_dict)

                # update information
                loss_acc += loss_value
                size_acc += batch_size
                total_acc+=acc
                pbar.update(batch_size)
                pbar.set_postfix(loss=loss_value)


        val_avg_loss = loss_acc / size_acc
        total_avg_acc=total_acc/size_acc
        if is_test:
            print(total_avg_acc)
        pbar.set_postfix(val_avg_loss=val_avg_loss)
        if is_need_log:
            pbar.close()  # destroy newly created pbar
            print('=' * 30 + ' Measurements ' + '=' * 30)
            for k, v in metrics_acc.items():
                print(f"[{k}] {v / size_acc}")
        else:
            return val_avg_loss,total_avg_acc,TP,TN,FP,FN,tpr_list, fpr_list

    def _aggregate_metrics(self, metrics_acc: dict, step_dict: dict):
        batch_size = step_dict['batch_size']
        for k, v in step_dict.items():
            if k[:7] == 'metric_':
                value = v * batch_size
                metric_name = k[7:]
                if metric_name not in metrics_acc:
                    metrics_acc[metric_name] = value
                else:
                    metrics_acc[metric_name] += value
        return metrics_acc


    def get_recorder(self) -> dict:
        return self.recorder




/tmp/ipykernel_9765/141488698.py:10: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm  # may raise warning about Jupyter


In [215]:
# import os
# import itertools
# import statistics
# from typing import Callable

# import numpy as np

# # from tqdm import tqdm
# from matplotlib import pyplot as plt
# from tqdm.autonotebook import tqdm  # may raise warning about Jupyter
# from tqdm.auto import tqdm  # who needs warnings

# import torch, torchvision
# from torch import nn
# from torch.utils import data as Data
# class Solver(object):
#     def __init__(self,
#                  model: nn.Module,
#                  optimizer: torch.optim.Optimizer,
#                  criterion: Callable,
#                  lr_scheduler = None,
#                  recorder: dict = None,
#                  device=None):
#         device = device if device is not None else \
#             ('cuda:0' if torch.cuda.is_available() else 'cpu')
#         self.device = device
#         self.recorder = recorder
        
#         self.model = self.to_device(model)
#         self.optimizer = optimizer
#         self.criterion = criterion
#         self.lr_scheduler = lr_scheduler        

#     def _step(self,
#               batch: tuple) -> dict:
#         raise NotImplementedError()

#     def to_device(self, x):
#         if isinstance(x, torch.Tensor):
#             return x.to(self.device)
#         elif isinstance(x, np.ndarray):
#             return torch.tensor(x, device=self.device)
#         elif isinstance(x, nn.Module):
#             return x.to(self.device)
#         else:
#             raise RuntimeError("Data cannot transfer to correct device.")

#     def to_numpy(self, x):
#         if isinstance(x, np.ndarray):
#             return x
#         elif isinstance(x, torch.Tensor):
#             return x.detach().cpu().numpy()
#         else:
#             raise RuntimeError(f"Cannot convert type {type(x)} into numpy array.")

#     def train(self,
#               epochs: int,
#               data_loader,
#               *,
#               val_loader=None,
#               is_plot=True) -> dict:             
#         torch.cuda.empty_cache()

#         val_loss_epochs = []
#         train_loss_epochs = []
#         val_acc=[]

#         pbar_train = tqdm(total=len(data_loader)*2-1, unit='block')
#         if val_loader is not None:
#             pbar_val = tqdm(total=len(val_loader), desc=f'[Validation] waiting', unit='block')

#         train_acc=[]
#         for epoch in range(epochs):

#             pbar_train.reset()
#             pbar_train.set_description(desc=f'[Train] Epoch {epoch + 1}/{epochs}')
#             epoch_loss_acc = 0
#             epoch_size = 0
#             epoch_acc=0
    
            
#             for batch in data_loader:
#                 self.model.train()
#                 # forward
#                 step_dict = self._step(batch)
#                 batch_size = step_dict['batch_size']
#                 loss = step_dict['loss']
#                 acc=step_dict['acc']
#                 # backward
#                 self.optimizer.zero_grad()
#                 loss.backward()

#                 # optimize
#                 self.optimizer.step()

#                 # update information
#                 loss_value = loss.item()
#                 epoch_loss_acc += loss_value
#                 epoch_size += batch_size
#                 epoch_acc+=acc

#                 pbar_train.update(batch_size)
#                 pbar_train.set_postfix(loss=loss_value / batch_size)

#             epoch_avg_loss = epoch_loss_acc / epoch_size
#             epoch_avg_acc=epoch_acc/epoch_size
#             pbar_train.set_postfix(epoch_avg_loss=epoch_avg_loss)
#             train_loss_epochs.append(epoch_avg_loss)
#             train_acc.append(epoch_avg_acc)

#             if self.lr_scheduler:
#                 self.lr_scheduler.step()

#             # validate if `val_loader` is specified
#             if val_loader is not None:
#                 pbar_val.reset()
#                 pbar_val.set_description(desc=f'[Validation] Epoch {epoch + 1}/{epochs}')
#                 val_avg_loss ,total_avg_acc= self.validate(val_loader, pbar=pbar_val,is_test=False)
#                 val_loss_epochs.append(val_avg_loss)
#                 val_acc.append(total_avg_acc)

#         pbar_train.close()
#         if val_loader is not None:
#             pbar_val.close()
#         train_loss_epochs = torch.tensor(train_loss_epochs).numpy()
#         val_loss_epochs = torch.tensor(val_loss_epochs).numpy()

#         plt.figure()
#         plt.plot(list(range(1, epochs + 1)), train_loss_epochs, label='train')
#         if val_loader is not None:
#             plt.plot(list(range(1, epochs + 1)), val_loss_epochs, label='validation')
#         plt.legend()
#         plt.xlabel('Epochs')
#         plt.ylabel('Loss')
#         plt.show()
#         plt.close('all')

#         if val_loader is not None:
#             train_acc=torch.tensor(train_acc).numpy()
#             val_acc=torch.tensor(val_acc).numpy()
#             plt.figure()
#             plt.plot(list(range(1, epochs + 1)), train_acc, label='train')
#             plt.plot(list(range(1, epochs + 1)), val_acc, label='validation')
#             plt.legend()
#             plt.xlabel('Epochs')
#             plt.ylabel('ACC')
#             plt.show()
#             plt.close('all')
#         print(self.model)
#         torch.save(self.model, 'ttttmodel.pth')
        


#     def validate(self, data_loader, *, pbar=None,is_test=False) -> float:
#         """
#         :param pbar: when pbar is specified, do not print average loss
#         :return:
#         """       
#         torch.cuda.empty_cache()

#         metrics_acc = {}
#         loss_acc = 0
#         size_acc = 0
#         total_acc=0
#         is_need_log = (pbar is None)        
#         with torch.no_grad():
#             if pbar is None:
#                 pbar = tqdm(total=len(data_loader), desc=f'[Validation]', unit='block')

            
#             for batch in data_loader:
#                 self.model.eval()

#                 # forward
#                 pred = self.model(batch[0].to(self.device))
#                 pred_mask=torch.sigmoid(pred)
#                 # print(batch[1],pred_mask)
#                 pred_mask[pred_mask>0.5]=1
#                 pred_mask[pred_mask<=0.5]=0
#                 acc=torch.sum(pred_mask==batch[1].to(torch.int).to(self.device))
#                 # print(acc)
#                 step_dict = self._step(batch)
#                 batch_size = step_dict['batch_size']
#                 loss = step_dict['loss']
#                 acc=step_dict['acc']
#                 # print(acc)
#                 loss_value = loss.item()

#                 # aggregate metrics
#                 metrics_acc = self._aggregate_metrics(metrics_acc, step_dict)

#                 # update information
#                 loss_acc += loss_value
#                 size_acc += batch_size
#                 total_acc+=acc
#                 pbar.update(batch_size)
#                 pbar.set_postfix(loss=loss_value)


#         val_avg_loss = loss_acc / size_acc
#         total_avg_acc=total_acc/size_acc
#         if is_test:
#             print(total_avg_acc)
#         pbar.set_postfix(val_avg_loss=val_avg_loss)
#         if is_need_log:
#             pbar.close()  # destroy newly created pbar
#             print('=' * 30 + ' Measurements ' + '=' * 30)
#             for k, v in metrics_acc.items():
#                 print(f"[{k}] {v / size_acc}")
#         else:
#             return val_avg_loss,total_avg_acc

#     def _aggregate_metrics(self, metrics_acc: dict, step_dict: dict):
#         batch_size = step_dict['batch_size']
#         for k, v in step_dict.items():
#             if k[:7] == 'metric_':
#                 value = v * batch_size
#                 metric_name = k[7:]
#                 if metric_name not in metrics_acc:
#                     metrics_acc[metric_name] = value
#                 else:
#                     metrics_acc[metric_name] += value
#         return metrics_acc


#     def get_recorder(self) -> dict:
#         return self.recorder




In [216]:
# import torch
# import torch.nn.functional as F
# import tensorflow as tf

# class LabSolver(Solver):
#     def _step(self, batch) -> dict:
#         block, gt = batch

#         block = self.to_device(block)  # [B, C=1, H, W]
#         gt = self.to_device(gt)  # [B, C=1, H, W]
        
#         B, C, H, W = block.shape

#         pred = self.model(block)  # [B, C=1, H, W]
#         gt = gt.view(B,1)
#         loss = self.criterion(pred, gt.to(torch.float32))

#         threshold=0.5
#         pred_mask=torch.sigmoid(pred)
#         pred_mask[pred_mask>threshold]=1
#         pred_mask[pred_mask<=threshold]=0
#         acc=torch.sum(pred_mask==gt.to(torch.int))
#         # mask=torch.argmax(pred_mask,dim=1)
#         # one_hot_pred = F.one_hot(mask, num_classes=pred_mask.shape[1])

#         # assert one_hot_pred.shape==gt.shape
#         # acc=torch.sum(one_hot_pred==gt)/2
    

#         step_dict = {
#             'loss': loss,
#             'batch_size': B,
#             'acc':acc
#         }

#         return step_dict



In [20]:
import torch
import torch.nn.functional as F

class LabSolver(Solver):
    def _step(self, batch) -> dict:
        block, gt = batch

        block = self.to_device(block)  # [B, C=1, H, W]
        gt = self.to_device(gt)  # [B, C=1, H, W]
        
        B, C, H, W = block.shape

        pred = self.model(block)  # [B, C=1, H, W]
        gt = gt.view(B,1)
        loss = self.criterion(pred, gt.to(torch.float32))

        threshold=0.5
        pred_mask=torch.sigmoid(pred)
        pred_mask[pred_mask>threshold]=1
        pred_mask[pred_mask<=threshold]=0
        acc=torch.sum(pred_mask==gt.to(torch.int))
        # mask=torch.argmax(pred_mask,dim=1)
        # one_hot_pred = F.one_hot(mask, num_classes=pred_mask.shape[1])

        # assert one_hot_pred.shape==gt.shape
        # acc=torch.sum(one_hot_pred==gt)/2
    

        step_dict = {
            'loss': loss,
            'batch_size': B,
            'acc':acc
        }

        return step_dict



In [218]:
# net=EcogBandRes()
# optimizer = torch.optim.Adam(net.parameters(), lr=0.5)
# lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.95)

# solver = LabSolver(
#     model = net,
#     optimizer = optimizer,
#     criterion = nn.BCEWithLogitsLoss(),
#     lr_scheduler=lr_scheduler
# )
# print(net)

In [21]:
net=EcogBandRes()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.95)

solver = LabSolver(
    model = net,
    optimizer = optimizer,
    criterion = nn.BCEWithLogitsLoss(),
    lr_scheduler=lr_scheduler
)
print(net)

/root/anaconda3/envs/ecog/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


EcogBandRes(
  (res): ECOGRes50(
    (res50): ResNet(
      (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=

In [22]:
solver.train(
    epochs=2, 
    data_loader=train_loader,
    val_loader=val_loader
)




NameError: name 'train_loader' is not defined

In [230]:

solver.validate(
    data_loader=val_loader,
    is_test=True
)





































































































































































































































































































































































































































[Validation]: 100%|██████████| 120/120 [00:02<00:00, 57.14block/s, val_avg_loss=3.74]

tensor(0.5000, device='cuda:0')
============================== Measurements ==============================


In [231]:
model_load=torch.load('sig_else2_model.pth')
solver.model = model_load
solver.validate(
    data_loader=val_loader,
    is_test=True

)











































































































































































































































































































































































































































[Validation]: 100%|██████████| 120/120 [00:02<00:00, 50.17block/s, val_avg_loss=1.23]

tensor(0.5000, device='cuda:0')
============================== Measurements ==============================


In [ ]:
# solver.validate(
#     data_loader=test_loader,
# #     is_test=True
    
# # )
# for batch in test_loader:
#     print(solver.model(batch[0].to(solver.device)))